# Backtrader: Quick Start

https://www.backtrader.com/docu/quickstart/quickstart/

## Basic Setup

In [47]:
from __future__ import (absolute_import, division, print_function, unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

import pandas as pd

In [3]:
if __name__ == '__main__':
    cerebro = bt.Cerebro() # The Cerebro engine was instantiated
    print('Starting portfolio value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.run() # The resulting cerebro instance was told to run (loop over data)
    print('Final portfolio value: %.2f' % cerebro.broker.getvalue())

Starting portfolio value: 10000.00
Final portfolio value: 10000.00


Although it doesn’t seem much, let’s point out something explicitly shown:

The Cerebro engine has created a broker instance in the background

The instance already has some cash to start with

This behind the scenes broker instantiation is a constant trait in the platform to simplify the life of the user. If no broker is set by the user, a default one is put in place.

And 10K monetary units is a usual value with some brokers to begin with.

## Setting the cash

In [4]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(1000000.0) # Set custom start cash. 
    print('Starting portfolio value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.run() 
    print('Final portfolio value: %.2f' % cerebro.broker.getvalue())

Starting portfolio value: 1000000.00
Final portfolio value: 1000000.00


## Adding a Data Feed

Having cash is fun, but the purpose behind all this is to let an automated strategy multiply the cash without moving a finger by operating on an asset which we see as a Data Feed

In [48]:
df = pd.read_csv("AAPL.csv")
df.head()

,date,open,high,low,close,volume
0,12/31/2014,112.820000,113.129997,110.209999,101.419060,41403400
1,1/2/2015,111.389999,111.440002,107.349999,100.454300,53204600
2,1/5/2015,108.290001,108.650001,105.410004,97.624336,64285500
3,1/6/2015,106.540001,107.430000,104.629997,97.633545,65797100
4,1/7/2015,107.199997,108.199997,106.699997,99.002556,40105900


In [49]:
'2011/23/11'.replace('/', '-')

'2011-23-11'

In [63]:
df.date = df.date.apply(lambda x: x.replace('/', '-'))
df.date = pd.to_datetime(df.date)
df.head() # yyyy-mm-dd 형태로 ohlcv 인 yahoo finance data format에 맞춰줌. 

,date,open,high,low,close,volume
0,2014-12-31,112.820000,113.129997,110.209999,101.419060,41403400
1,2015-01-02,111.389999,111.440002,107.349999,100.454300,53204600
2,2015-01-05,108.290001,108.650001,105.410004,97.624336,64285500
3,2015-01-06,106.540001,107.430000,104.629997,97.633545,65797100
4,2015-01-07,107.199997,108.199997,106.699997,99.002556,40105900


In [65]:
df.to_csv('AAPL.csv', index=0)

In [66]:
os.getcwd()

'C:\\Users\\chlje\\VSCodeProjects\\finance_essentials_37-64'

In [67]:
datapath = os.path.join(os.getcwd(), './AAPL.csv')
datapath

'C:\\Users\\chlje\\VSCodeProjects\\finance_essentials_37-64\\./AAPL.csv'

In [68]:
os.path.isfile(datapath)

True

In [69]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    
#     modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
#     datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')
    datapath = os.path.join(os.getcwd(), './AAPL.csv')

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2000, 1, 1),
        # Do not pass values after this date
        todate=datetime.datetime(2000, 12, 31),
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
